In [ ]:
%load_ext autoreload
%autoreload 2
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from braininversion.BiotSolverIterative import solve_timedep_biot_iterative
import sympy as sym
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, compute_order)
set_log_level(40)

u_degree = 2
p_degree = 1 
resolutions = [5,10,20,30,40]   # resolution

expr_degree = 2
T = 10          # final time
num_steps = 5    # number of time steps
dt = T / num_steps


material_parameter = dict()
material_parameter["c"] = 1
material_parameter["K"] = 1
material_parameter["lmbda"] = 1e12
material_parameter["mu"] = 1
material_parameter["alpha"] = 1.0

boundary_all = CompiledSubDomain("on_boundary")
boundary_right = CompiledSubDomain("on_boundary && near(x[0], 1)")
boundary_left = CompiledSubDomain("on_boundary && near(x[0], 0)")
boundary_top = CompiledSubDomain("on_boundary && near(x[1], 1)")
boundary_bottom = CompiledSubDomain("on_boundary && near(x[1], 0)")

In [ ]:
from sympy import symbols, pi
from sympy import derive_by_array, tensorcontraction

c, mu, lmbda, alpha, t, K = sym.symbols('c, mu, lmbda, alpha, t, K')
K = material_parameter["K"]

x,y,z = sym.symbols("x, y, z")

ux = (sym.sin(4*pi*x) + sym.sin(3*pi*y))
uy = (sym.cos(2*pi*x) + sym.sin(5*pi*y))
ux = 2*x - 8*y*y
uy = 2*x*y - 8*y

u = sym.Matrix([ux, uy, 0])*t
p = (0.1*x - y + 6*y)*t
#p = (sym.cos(3*pi*x) + sym.sin(1*pi*y))*t

def row_wise_div(tensor):
    return sym.Matrix(tensorcontraction(derive_by_array( tensor, [x,y,z]), (0,2)))
    
def row_wise_grad(u):
    return u.jacobian([x,y,z])

def eps(u):
    return 0.5*(row_wise_grad(u) + sym.transpose(row_wise_grad(u)))

def div(u):
    return sym.trace(u.jacobian([x,y,z]))

def grad(p):
    return sym.Matrix([sym.diff(p,x), sym.diff(p,y), sym.diff(p,z)])
 
f = - row_wise_div(mu*eps(u) + lmbda*div(u)*sym.eye(3) - alpha*p*sym.eye(3))
g = + (alpha**2)/lmbda*sym.diff(p, t) + alpha*div(sym.diff(u, t)) - div(K*grad(p))

f.simplify()
g.simplify()

p_N_right = - sym.diff( p, x).subs(x, 1)
p_N_left = sym.diff(p, x).subs(x, 0)
p_N_bottom = sym.diff(p, y).subs(y, 0)
p_N_top = - sym.diff(p, y).subs(y, 1)


fx, fy = f[0], f[1]
ux, uy = u[0], u[1]

xcpp, ycpp = sym.symbols('x[0], x[1]')

variables_scalar = [g, p, p_N_right, p_N_left, p_N_bottom, p_N_top]
variables_scalar_cpp  = [sym.printing.ccode(v.subs(x, xcpp).subs(y, ycpp)) for v in variables_scalar]
variables_scalar_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_scalar_cpp]
g, p_e, p_N_right, p_N_left, p_N_bottom, p_N_top = variables_scalar_expr


variables_vector = [f, u]
variables_vector_cpp  = [(sym.printing.ccode(v[0].subs(x, xcpp).subs(y, ycpp)),
                          sym.printing.ccode(v[1].subs(x, xcpp).subs(y, ycpp)))
                          for v in variables_vector]
variables_vector_expr  = [Expression(v, **material_parameter, t=0, degree=expr_degree)
                          for v in variables_vector_cpp]
f, u_e = variables_vector_expr

In [ ]:
solution = None
mesh = None
solver_params={"relativeconv":False, "tolerance":1e-32, "maxiter":5000}
def biot_convergence(resolutions, boundaries,
                     boundary_conditions_p, boundary_conditions_u):
    global mesh
    global solution
    L2_errors_u = []
    L2_errors_p = []
    h = []
    for N in resolutions:
        p_e.t = 0
        mesh = UnitSquareMesh(N, N)
        h.append(mesh.hmax())

        boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
        
        for b, marker_id in boundaries.items():
            b.mark(boundary_marker, marker_id)
            
        solution = solve_timedep_biot_iterative(mesh, f, g, T, num_steps, material_parameter,
                         boundary_marker, boundary_conditions_p,
                         boundary_marker, boundary_conditions_u,
                         #p_initial=p_e, u_initial=u_e,
                         u_degree=u_degree, p_degree=p_degree,solver_params=solver_params
                         )
        solution = [s for s in solution]
        J_u = 0
        J_p = 0
        for i,up in enumerate(solution):
            u, pT, p = up
            p_e.t = (i + 1)*dt
            u_e.t = (i + 1)*dt
            J_p += errornorm(p_e, p, "L2")
            J_u += errornorm(u_e, u, "L2")

        L2_errors_u.append(J_u/num_steps)
        L2_errors_p.append(J_p/num_steps)

    return L2_errors_u, L2_errors_p, h, solution    


In [ ]:
# Neumann boundary conditions
boundaries = {boundary_top: 1,
              boundary_bottom: 1,
              boundary_left: 1,
              boundary_right: 1}

boundary_conditions_p = {1:{"Dirichlet": p_e},
                         #2:{"Neumann": p_N_left},
                         #3:{"Neumann": p_N_right},
                         }

boundary_conditions_u = {1:{"Dirichlet":u_e},
                         #2:{"Dirichlet":u_e},
                         #3:{"Dirichlet":u_e}
                        }

res = biot_convergence(resolutions, boundaries, 
                       boundary_conditions_p, boundary_conditions_u)
L2_errors_u_neum, L2_errors_p_neum, h, solution = res  

print(compute_order(L2_errors_u_neum, h))
print(compute_order(L2_errors_p_neum, h))

In [ ]:
L2_errors_p_neum

In [ ]:
L2_errors_u_neum

In [ ]:
i = 0
p_e.t = dt*(i+1)
p_h = solution[i][2]
c = plot(p_h - p_e, mesh=mesh)
plt.colorbar(c)

In [ ]:
i = 0
u_e.t = dt*(i+1)
u_h = solution[i][0]
c = plot(u_e - u_h, mesh=mesh)
plt.colorbar(c)